In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
train = pd.read_csv("./data/8-1 train(std).csv",index_col=0)
test1 = pd.read_csv("./data/8-1 test1(std).csv",index_col=0)
test2 = pd.read_csv("./data/8-1 test2(std).csv",index_col=0)


In [3]:
x_train = train.drop("label",axis=1)
y_train = train["label"]
x_test =test1.drop("label",axis=1)
y_test = test1["label"]

In [4]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [5]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")



In [6]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [9]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()

lda_clf.fit(x_train, y_train)
lda_pred = lda_clf.predict(x_test)
lda_pred_proba = lda_clf.predict_proba(x_test)
print("---------------------------------------------------------") 
get_clf_eval(y_test, lda_pred)
get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)
    

---------------------------------------------------------
오차행렬
[[2676   50]
 [  20   11]]
정확도: 0.9746, 정밀도 : 0.1803, 재현율:0.3548,F1 스코어:0.2391
ROC 스코어: 0.6682, PR score : 0.0712
임곗값: 0.1
오차행렬
[[2606  120]
 [  18   13]]
정확도: 0.9499, 정밀도 : 0.0977, 재현율:0.4194,F1 스코어:0.1585
ROC 스코어: 0.6877, PR score : 0.0475
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2634   92]
 [  19   12]]
정확도: 0.9597, 정밀도 : 0.1154, 재현율:0.3871,F1 스코어:0.1778
ROC 스코어: 0.6767, PR score : 0.0516
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2653   73]
 [  20   11]]
정확도: 0.9663, 정밀도 : 0.1310, 재현율:0.3548,F1 스코어:0.1913
ROC 스코어: 0.6640, PR score : 0.0537
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2661   65]
 [  20   11]]
정확도: 0.9692, 정밀도 : 0.1447, 재현율:0.3548,F1 스코어:0.2056
ROC 스코어: 0.6655, PR score : 0.0586
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2676   50]
 [  20   11]]
정확도: 0.9746, 정밀도 : 0.1803, 재현율:0.3548,

In [11]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)

dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
dt_pred_proba = dt_clf.predict_proba(x_test)   

get_clf_eval(y_test, dt_pred)
get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


오차행렬
[[2683   43]
 [  25    6]]
정확도: 0.9753, 정밀도 : 0.1224, 재현율:0.1935,F1 스코어:0.1500
ROC 스코어: 0.5889, PR score : 0.0328
임곗값: 0.1
오차행렬
[[2683   43]
 [  25    6]]
정확도: 0.9753, 정밀도 : 0.1224, 재현율:0.1935,F1 스코어:0.1500
ROC 스코어: 0.5889, PR score : 0.0328
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2683   43]
 [  25    6]]
정확도: 0.9753, 정밀도 : 0.1224, 재현율:0.1935,F1 스코어:0.1500
ROC 스코어: 0.5889, PR score : 0.0328
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2683   43]
 [  25    6]]
정확도: 0.9753, 정밀도 : 0.1224, 재현율:0.1935,F1 스코어:0.1500
ROC 스코어: 0.5889, PR score : 0.0328
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2683   43]
 [  25    6]]
정확도: 0.9753, 정밀도 : 0.1224, 재현율:0.1935,F1 스코어:0.1500
ROC 스코어: 0.5889, PR score : 0.0328
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2683   43]
 [  25    6]]
정확도: 0.9753, 정밀도 : 0.1224, 재현율:0.1935,F1 스코어:0.1500
ROC 스코어: 0.5889, PR score : 0.0328
---------

In [15]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)

# key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
param_grid = {
    'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10]}
grid_search = GridSearchCV(tree
                        , param_grid = param_grid
                        , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                        , n_jobs = -1)  #-1: 모든 코어 사용

grid_search.fit(x_train, y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
dt_clf_best.fit(x_train, y_train)
dt_best_pred = dt_clf_best.predict(x_test)
dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

print("디시전트리 파라미터적용") 
get_clf_eval(y_test, dt_best_pred)
get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 8, 'min_samples_split': 4}, 최적 평균 정확도 :0.985
디시전트리 파라미터적용
오차행렬
[[2715   11]
 [  26    5]]
정확도: 0.9866, 정밀도 : 0.3125, 재현율:0.1613,F1 스코어:0.2128
ROC 스코어: 0.5786, PR score : 0.0598
임곗값: 0.1
오차행렬
[[2702   24]
 [  24    7]]
정확도: 0.9826, 정밀도 : 0.2258, 재현율:0.2258,F1 스코어:0.2258
ROC 스코어: 0.6085, PR score : 0.0597
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2702   24]
 [  24    7]]
정확도: 0.9826, 정밀도 : 0.2258, 재현율:0.2258,F1 스코어:0.2258
ROC 스코어: 0.6085, PR score : 0.0597
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2715   11]
 [  26    5]]
정확도: 0.9866, 정밀도 : 0.3125, 재현율:0.1613,F1 스코어:0.2128
ROC 스코어: 0.5786, PR score : 0.0598
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2715   11]
 [  26    5]]
정확도: 0.9866, 정밀도 : 0.3125, 재현율:0.1613,F1 스코어:0.2128
ROC 스코어: 0.5786, PR score : 0.0598
------------------------------------------

In [16]:
print("디시전트리") 
print(get_clf_eval(y_test, lda_pred))

디시전트리
오차행렬
[[2676   50]
 [  20   11]]
정확도: 0.9746, 정밀도 : 0.1803, 재현율:0.3548,F1 스코어:0.2391
ROC 스코어: 0.6682, PR score : 0.0712
None


In [19]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)
print("랜덤포레스트") 
get_clf_eval(y_test, rf_pred)
get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)



랜덤포레스트
오차행렬
[[2722    4]
 [  28    3]]
정확도: 0.9884, 정밀도 : 0.4286, 재현율:0.0968,F1 스코어:0.1579
ROC 스코어: 0.5477, PR score : 0.0516
임곗값: 0.1
오차행렬
[[2612  114]
 [  14   17]]
정확도: 0.9536, 정밀도 : 0.1298, 재현율:0.5484,F1 스코어:0.2099
ROC 스코어: 0.7533, PR score : 0.0762
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2672   54]
 [  21   10]]
정확도: 0.9728, 정밀도 : 0.1562, 재현율:0.3226,F1 스코어:0.2105
ROC 스코어: 0.6514, PR score : 0.0580
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2694   32]
 [  24    7]]
정확도: 0.9797, 정밀도 : 0.1795, 재현율:0.2258,F1 스코어:0.2000
ROC 스코어: 0.6070, PR score : 0.0492
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2714   12]
 [  26    5]]
정확도: 0.9862, 정밀도 : 0.2941, 재현율:0.1613,F1 스코어:0.2083
ROC 스코어: 0.5784, PR score : 0.0569
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2722    4]
 [  28    3]]
정확도: 0.9884, 정밀도 : 0.4286, 재현율:0.0968,F1 스코어:0.1579
ROC 스코어: 0.5477, PR score : 0.0516
--

In [20]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의

grid_search = {'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10],
            'n_estimators': [20,40,60,80,100]}

model = RandomForestClassifier(random_state=52)
grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                            cv = 5,verbose=0, n_jobs = -1)
grid_search.fit(x_train,y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
rf_clf_best.fit(x_train, y_train)
rf_best_pred = rf_clf_best.predict(x_test)
rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

print("랜덤포레스트 파라미터적용") 
get_clf_eval(y_test, rf_best_pred)
get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 40}, 최적 평균 정확도 :0.986
랜덤포레스트 파라미터적용
오차행렬
[[2723    3]
 [  27    4]]
정확도: 0.9891, 정밀도 : 0.5714, 재현율:0.1290,F1 스코어:0.2105
ROC 스코어: 0.5640, PR score : 0.0835
임곗값: 0.1
오차행렬
[[2647   79]
 [  20   11]]
정확도: 0.9641, 정밀도 : 0.1222, 재현율:0.3548,F1 스코어:0.1818
ROC 스코어: 0.6629, PR score : 0.0506
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2682   44]
 [  22    9]]
정확도: 0.9761, 정밀도 : 0.1698, 재현율:0.2903,F1 스코어:0.2143
ROC 스코어: 0.6371, PR score : 0.0573
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2699   27]
 [  24    7]]
정확도: 0.9815, 정밀도 : 0.2059, 재현율:0.2258,F1 스코어:0.2154
ROC 스코어: 0.6080, PR score : 0.0552
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2715   11]
 [  25    6]]
정확도: 0.9869, 정밀도 : 0.3529, 재현율:0.1935,F1 스코어:0.2500
ROC 스코어: 0.5948, PR score : 0.0774
--------------------

In [22]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)
print("로지스틱회귀") 
get_clf_eval(y_test, lr_pred)
get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

로지스틱회귀
오차행렬
[[2713   13]
 [  25    6]]
정확도: 0.9862, 정밀도 : 0.3158, 재현율:0.1935,F1 스코어:0.2400
ROC 스코어: 0.5944, PR score : 0.0702
임곗값: 0.1
오차행렬
[[2653   73]
 [  18   13]]
정확도: 0.9670, 정밀도 : 0.1512, 재현율:0.4194,F1 스코어:0.2222
ROC 스코어: 0.6963, PR score : 0.0699
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2688   38]
 [  20   11]]
정확도: 0.9790, 정밀도 : 0.2245, 재현율:0.3548,F1 스코어:0.2750
ROC 스코어: 0.6704, PR score : 0.0869
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2699   27]
 [  22    9]]
정확도: 0.9822, 정밀도 : 0.2500, 재현율:0.2903,F1 스코어:0.2687
ROC 스코어: 0.6402, PR score : 0.0806
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2708   18]
 [  23    8]]
정확도: 0.9851, 정밀도 : 0.3077, 재현율:0.2581,F1 스코어:0.2807
ROC 스코어: 0.6257, PR score : 0.0877
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2713   13]
 [  25    6]]
정확도: 0.9862, 정밀도 : 0.3158, 재현율:0.1935,F1 스코어:0.2400
ROC 스코어: 0.5944, PR score : 0.0702
--

In [23]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV


params = {'penalty': ['l2','l1','elasticnet','None'],
        'C':[0.01,0.1,1,3,5,10]}

grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_search.fit(x_train,y_train)
print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
lr_clf_best.fit(x_train, y_train)
lr_best_pred = lr_clf_best.predict(x_test)
lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

print("로지스틱회귀 파라미터적용") 
get_clf_eval(y_test, lr_best_pred)
get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미어:{'C': 0.1, 'penalty': 'l2'}, 최적 평균 정확도:0.982
로지스틱회귀 파라미터적용
오차행렬
[[2712   14]
 [  25    6]]
정확도: 0.9859, 정밀도 : 0.3000, 재현율:0.1935,F1 스코어:0.2353
ROC 스코어: 0.5942, PR score : 0.0671
임곗값: 0.1
오차행렬
[[2655   71]
 [  18   13]]
정확도: 0.9677, 정밀도 : 0.1548, 재현율:0.4194,F1 스코어:0.2261
ROC 스코어: 0.6967, PR score : 0.0714
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2690   36]
 [  20   11]]
정확도: 0.9797, 정밀도 : 0.2340, 재현율:0.3548,F1 스코어:0.2821
ROC 스코어: 0.6708, PR score : 0.0903
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2701   25]
 [  22    9]]
정확도: 0.9830, 정밀도 : 0.2647, 재현율:0.2903,F1 스코어:0.2769
ROC 스코어: 0.6406, PR score : 0.0848
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2706   20]
 [  24    7]]
정확도: 0.9840, 정밀도 : 0.2593, 재현율:0.2258,F1 스코어:0.2414
ROC 스코어: 0.6092, PR score : 0.0672
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2712   14]
 [  25    6]]
정확도: 0.9859, 정밀도 : 0.3000